<a href="https://colab.research.google.com/github/sunn-123/PANDA-rerlated-project/blob/main/stock_predection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, mean_squared_error

# Load and clean data
file_path ='/content/National_Stock_Exchange_of_India_Ltd.csv'
df = pd.read_csv(file_path)

# Clean numeric columns
cols_to_clean = ['Open', 'High', 'Low', 'LTP', 'Chng', '% Chng', 'Volume (lacs)',
                 'Turnover (crs.)', '52w H', '52w L', '365 d % chng', '30 d % chng']
for col in cols_to_clean:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

df.dropna(inplace=True)

# Feature Engineering
df['Price_Range'] = df['High'] - df['Low']
df['Daily_Return'] = df['Chng'] / df['Open']
df['Volatility'] = df['Price_Range'] / df['Open']

# -----------------------------
# 1. Price Direction Prediction (Up/Down)
# -----------------------------
df['Price_Up'] = (df['Chng'] > 0).astype(int)
X1 = df[['Open', 'High', 'Low', 'Price_Range', 'Daily_Return', 'Volatility']]
y1 = df['Price_Up']

scaler1 = StandardScaler()
X1_scaled = scaler1.fit_transform(X1)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1_scaled, y1, test_size=0.2, random_state=42)
clf1 = RandomForestClassifier(random_state=42)
clf1.fit(X1_train, y1_train)
y1_pred = clf1.predict(X1_test)

print("🔹 [1] Price Direction Prediction (Up/Down)")
print(classification_report(y1_test, y1_pred, zero_division=0))

# -----------------------------
# 2. Next Day Price Prediction (Regression)
# -----------------------------
X2 = X1  # reuse engineered features
y2 = df['LTP']

scaler2 = StandardScaler()
X2_scaled = scaler2.fit_transform(X2)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2_scaled, y2, test_size=0.2, random_state=42)
reg2 = RandomForestRegressor(random_state=42)
reg2.fit(X2_train, y2_train)
y2_pred = reg2.predict(X2_test)

print("🔹 [2] Closing Price Prediction")
print("MSE:", mean_squared_error(y2_test, y2_pred))

# -----------------------------
# 3. Volatility Prediction (Classification)
# -----------------------------
df['Volatile'] = ((df['High'] - df['Low']) > (0.03 * df['Open'])).astype(int)
X3 = X1
y3 = df['Volatile']

scaler3 = StandardScaler()
X3_scaled = scaler3.fit_transform(X3)

X3_train, X3_test, y3_train, y3_test = train_test_split(X3_scaled, y3, test_size=0.2, random_state=42)
clf3 = RandomForestClassifier(random_state=42)
clf3.fit(X3_train, y3_train)
y3_pred = clf3.predict(X3_test)

print("🔹 [3] Volatility Prediction")
print(classification_report(y3_test, y3_pred, zero_division=0))

# -----------------------------
# 4. Volume Prediction (Regression)
# -----------------------------
X4 = X1
y4 = df['Volume (lacs)']

scaler4 = StandardScaler()
X4_scaled = scaler4.fit_transform(X4)

X4_train, X4_test, y4_train, y4_test = train_test_split(X4_scaled, y4, test_size=0.2, random_state=42)
reg4 = RandomForestRegressor(random_state=42)
reg4.fit(X4_train, y4_train)
y4_pred = reg4.predict(X4_test)

print("🔹 [4] Volume Prediction")
print("MSE:", mean_squared_error(y4_test, y4_pred))


🔹 [1] Price Direction Prediction (Up/Down)
              precision    recall  f1-score   support

           0       0.90      1.00      0.95         9
           1       0.00      0.00      0.00         1

    accuracy                           0.90        10
   macro avg       0.45      0.50      0.47        10
weighted avg       0.81      0.90      0.85        10

🔹 [2] Closing Price Prediction
MSE: 25018958.129480194
🔹 [3] Volatility Prediction
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         6

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

🔹 [4] Volume Prediction
MSE: 4302.579326879002
